# JSON데이터 CSV 변환

In [ ]:
pip install pandas

In [ ]:
# 라이브러리 로드
import json
import logging
import pandas as pd

In [ ]:
# 로깅
logging.basicConfig(level=logging.ERROR,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    filename='error.log',
                    filemode='a')
logger = logging.getLogger(__name__)

In [ ]:
# 데이터 변환 메소드
def convert_raw_data_to_csv(file_name: str, id_selector: str, fields: dict, delimiter: str = None):
    '''
    ## json파일을 읽고 처리하여 csv파일로 저장하는 함수\n
    args: \n
    file_name: 파일명 \n
    id_selector: 전처리를위한 id 이름(work_id일경우 id_selector => work) \n
    fields: csv에 사용 될 필드명의 딕셔너리 \n
    delimiter: 구분자가 ','가 아닐시 원하는 delimiter. -> '|', '`'등 사용 가능하다 \n
    retrun: 첫행에 field가 적혀진 csv 파일
    '''

    # 파일 불러오기 및 저장
    input_file = file_name
    if delimiter:
        output_file = f'{file_name}'[:-5] + '_with_delimiter.csv'
    else:
        output_file = f'{file_name}'[:-5] + '.csv'

    # load 할 때 파일 크기가 클 경우 에러발생시 예외처리
    try:
        with open(input_file, 'r', encoding="utf-8") as f:
            raw_data = json.load(f)
    except Exception as e:
        print("파일 로드시 에러가 발생했습니다. 로그를 확인해주세요")
        logger.exception(str(e))
        return

    # csv 파일 생성 부분
    try:
        df = pd.json_normalize(raw_data)
        selected_fields = fields

        # 필드 존재 여부 확인 (없는 경우 무시)
        existing_fields = {k: v for k, v in selected_fields.items() if k in df.columns}

        # 선택한 필드만 추출
        filtered_df = df[list(existing_fields.keys())].rename(columns=existing_fields)

        # id 전처리 부분
        if f'{id_selector}_id' in filtered_df.columns:
            filtered_df[f'{id_selector}_id'] = filtered_df[f'{id_selector}_id'].astype(str).apply(lambda x: x[21:])
        else:
            filtered_df = df[list(existing_fields.keys())].rename(columns=existing_fields)

        # csv로 저장
        if delimiter:
            filtered_df.to_csv(output_file, index=False, sep=delimiter, encoding="utf-8-sig")
            print("delimiter 파일 생성 완료")
        else:
            filtered_df.to_csv(output_file, index=False)
            print("delimiter 없는 파일 생성 완료")

    except Exception as e:
        print("데이터 변환시 오류발생. 로그를 확인해주세요")
        logger.exception(str(e))
        
    return 

In [ ]:
# topic JSON 데이터 변환
file_name = "topic.json"
id_selector = "topic"
fields = {
    "id": "topic_id", 
    "display_name": "topic", 
    "domain.display_name": "domain", 
    "field.display_name": "filed", 
    "subfield.display_name": "subfield",
    "works_count": "works_count",
    "cited_by_count": "cited_by_count"
    }

In [ ]:
# Institution 데이터 변환
file_name = "institution.json"
id_selector = "institution"
fields = {
    "id": "institution_id",
    "display_name": "institution_name",
    "country_code": "country_code",
    "type": "type",
    "geo.latitude": "geo_lat",
    "geo.longitude": "geo_lon",
    "works_count": "works_count",
    "cited_by_count": "cited_by_count"
}

In [ ]:
convert_raw_data_to_csv(file_name=file_name, id_selector=id_selector, fields=fields)
# convert_raw_data_to_csv(file_name=file_name, id_selector=id_selector, fields=fields, delimiter='`')